In [1]:
import torch
import numpy as np
import open3d as o3d
from omegaconf import OmegaConf
import copy

from model_builder import ModelBuilder
from data import BagDataset
from util import get_colors

In [2]:
config = OmegaConf.load('conf.yml')
config.k = 9
# config.model_path = '/Users/nikhil.k/data/dev/lbh/databricks_models/best_model.ckpt'
# config.model_path = "/Users/nikhil.k/data/dev/lbh/databricks_models/mlflow_best.ckpt"
config.model_path = "/Users/nikhil.k/data/dev/lbh/databricks_models/no_floor.ckpt"

In [3]:
model = ModelBuilder.load_from_checkpoint(config=config, checkpoint_path=config.model_path)

In [4]:
parsed_data, pcd = BagDataset.parse_data(
    # "/Users/nikhil.k/data/dev/lbh/converted-bag/labels/test_labels/p12_2021-04-15-00-44-02.json"
    "/Users/nikhil.k/data/dev/lbh/converted-bag/labels/train_labels/p7_2021-04-15-00-50-02.json"
)
roi_pcd, ob_roi = BagDataset.crop_volume(pcd, parsed_data['roi'])
box_pcd, ob_box = BagDataset.crop_volume(pcd, parsed_data['box'])

roi_points, roi_colors, roi_normals = BagDataset.get_np_points(roi_pcd)
box_points, box_colors, box_normals = BagDataset.get_np_points(box_pcd)

labels_one_hot = BagDataset.get_labels(box_points, roi_points).unsqueeze(0)
# The shape #points, 9 (3-xyz, 3-rgb, 3-normals)
input_tensor = torch.zeros((1, roi_points.shape[0], 9))
input_tensor[0, :, 0:3] = torch.tensor(roi_points)
input_tensor[0, :, 3:6] = torch.tensor(roi_colors)
# if config.get("set_normals"):
    # input_tensor[0, 6:9, :] = torch.tensor(roi_normals.transpose())

TypeError: parse_data() missing 1 required positional argument: 'file_name'

In [ ]:
input_tensor = input_tensor.permute(0, 2, 1)
pred = model(input_tensor)
pred_all = pred.permute(0, 2, 1).contiguous()
pred = pred_all.max(dim=2)[1]

In [ ]:
def color_pcd(pred, pcd):
    colors = []
    pred_np = pred#.detach().numpy()[0]
    pred_colors = np.zeros((pred_np.shape[0], 3))
    sem_seg_colors = get_colors()
    for index in range(pred_np.shape[0]):
        color = pred_np[index]+1
        pred_colors[index, :] = np.array(sem_seg_colors[color])#/255.0

    predicted_pcd = copy.deepcopy(pcd)
    predicted_pcd.colors = o3d.utility.Vector3dVector(pred_colors/255.0)
    return predicted_pcd

In [ ]:
hull, _ = predicted_pcd.compute_convex_hull()
hull.get_volume()

0.0005464177646234784

In [ ]:
predicted_pcd = color_pcd(pred.detach().numpy()[0], roi_pcd)

In [ ]:
# hull, _ = box.compute_convex_hull()
# hull.orient_triangles()
# hull.compute_vertex_normals()
# box2 = hull.sample_points_poisson_disk(3000)
# lineset = o3d.geometry.LineSet.create_from_triangle_mesh(hull)


# pcd_t = copy.deepcopy(roi_pcd).translate((1.5, 0, 0))
# pred_t = copy.deepcopy(predicted_pcd).translate((2.5, 0, 0))
# box_t = copy.deepcopy(box).translate((3.5, 0, 0))
# pred_t_lineset = copy.deepcopy(predicted_pcd).translate((4.5, 0, 0))
# lineset_t = copy.deepcopy(lineset).translate((4.5, 0, 0))
# o3d.visualization.draw_geometries([pcd_t, pred_t, box_t, pred_t_lineset, lineset_t], window_name='')
# # o3d.visualization.draw_geometries([pred_t], window_name='')

In [ ]:
def generate_vis(pred, predicted_pcd, threshold=0.01):

    box = predicted_pcd.select_by_index(np.where(pred==1)[1])
    box_p = np.array(box.points)
    box_filtered = box.select_by_index(np.where(box_p[:, 2]>threshold)[0])


    # from sklearn.neighbors import LocalOutlierFactor
    # clf = LocalOutlierFactor(n_neighbors=2)
    # out_liers = clf.fit_predict(np.array(box_filtered.points))
    # box_filtered = box_filtered.select_by_index(np.where(out_liers==1)[0])


    hull, _ = box_filtered.compute_convex_hull()
    hull.orient_triangles()
    hull.compute_vertex_normals()
    lineset = o3d.geometry.LineSet.create_from_triangle_mesh(hull)

    points = np.array(predicted_pcd.points)
    x = predicted_pcd.select_by_index(np.where(points[:,2] > threshold)[0])


    pcd_t = copy.deepcopy(roi_pcd).translate((1.5, 0, 0))
    pred_t = copy.deepcopy(x).translate((2.5, 0, 0))
    box_t = copy.deepcopy(box).translate((3.5, 0, 0))
    pred_t_lineset = copy.deepcopy(x).translate((4.5, 0, 0))
    lineset_t = copy.deepcopy(lineset).translate((4.5, 0, 0))
    # o3d.visualization.draw_geometries([pcd_t, pred_t, box_t, pred_t_lineset, lineset_t, x], window_name='')
    o3d.visualization.draw_geometries([pred_t_lineset, lineset_t], window_name='')
    # o3d.visualization.draw_geometries([pred_t], window_name='')
    # return x

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(2, weights='distance')
clf.fit(np.array(roi_pcd.points), pred[0])
pred_nn = clf.predict(np.array(roi_pcd.points))

In [ ]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [ ]:
nn_pcd = color_pcd(pred.detach().numpy()[0], roi_pcd)
# o3d.visualization.draw_geometries([nn_pcd], window_name='')

In [ ]:
# output = generate_vis(pred.detach().numpy()[0], predicted_pcd)
# k_out = generate_vis(pred_nn, nn_pcd)

In [ ]:
cl, ind = nn_pcd.remove_statistical_outlier(nb_neighbors=2, std_ratio=1.0)
# cl, ind = predicted_pcd.remove_radius_outlier(nb_points=16, radius=0.05)
# display_inlier_outlier(nn_pcd, ind)

In [ ]:
generate_vis(pred_nn[ind], nn_pcd.select_by_index(ind))

IndexError: tuple index out of range

In [ ]:
generate_vis(pred.detach().numpy()[0][ind], predicted_pcd.select_by_index(ind))

In [ ]:
generate_vis(pred.detach().numpy(), predicted_pcd, 0)

In [ ]:
(predicted_pcd.get_max_bound() - predicted_pcd.get_min_bound())*100

array([86.52746081, 81.95562959, 37.92455457])